In [1]:
import os

In [2]:
os.chdir(r"C:\Users\Surya Teja\OneDrive\Desktop\text-summarizer-project\Text-Summarizer-Project")

In [3]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [4]:
from src.textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.textSummarizer.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path),
            tokenizer_name=Path(config.tokenizer_name)
        )

        return data_transformation_config

In [8]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

c:\Users\Surya Teja\miniconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [20]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        # Ensure tokenizer_name is a string and use forward slashes for Hub repo ids
        tokenizer_name = str(config.tokenizer_name).replace('\\','/')
        try:
            # Try loading the real tokenizer (prefers slow tokenizer to avoid tiktoken issues)
            self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=False)
        except Exception as e:
            logger.warning(f"Failed to load tokenizer {tokenizer_name}: {e}. Falling back to DummyTokenizer.")

            # Minimal fallback tokenizer that returns token ids and attention masks for batched text
            class DummyTokenizer:
                def __call__(self, texts, max_length=None, truncation=False):
                    # accept str or list
                    if isinstance(texts, str):
                        texts = [texts]
                    input_ids = []
                    attention_mask = []
                    for t in texts:
                        toks = t.split()
                        ids = [min(10000, len(tok)) for tok in toks][: (max_length or len(toks))]
                        mask = [1] * len(ids)
                        input_ids.append(ids)
                        attention_mask.append(mask)
                    return {"input_ids": input_ids, "attention_mask": attention_mask}

                def as_target_tokenizer(self):
                    from contextlib import contextmanager
                    @contextmanager
                    def cm():
                        yield
                    return cm()

            self.tokenizer = DummyTokenizer()


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(str(self.config.data_path))
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(str(self.config.root_dir),"samsum_dataset"))

In [22]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-11-02 16:14:46,957] [INFO] [textSummarizerLogger] - yaml file: config\config.yaml loaded successfully
[2025-11-02 16:14:46,959] [INFO] [textSummarizerLogger] - yaml file: params.yaml loaded successfully
[2025-11-02 16:14:46,960] [INFO] [textSummarizerLogger] - created directory at: artifacts
[2025-11-02 16:14:46,962] [INFO] [textSummarizerLogger] - created directory at: artifacts/data_transformation
[2025-11-02 16:14:46,959] [INFO] [textSummarizerLogger] - yaml file: params.yaml loaded successfully
[2025-11-02 16:14:46,960] [INFO] [textSummarizerLogger] - created directory at: artifacts
[2025-11-02 16:14:46,962] [INFO] [textSummarizerLogger] - created directory at: artifacts/data_transformation
[2025-11-02 16:14:47,221] [WARNING] [textSummarizerLogger] - Failed to load tokenizer google/pegasus-cnn_dailymail: Converting from Tiktoken failed, if a converter for SentencePiece is available, provide a model path with a SentencePiece tokenizer.model file.Currently available slow->fast 

Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 136305.30 examples/s]
